# Prueba A/B del sistema de recomendaciones

El proyecto consiste en una prueba A/B realizada por una tienda en línea internacional para evaluar la efectividad de un nuevo sistema de recomendaciones. La prueba se llevó a cabo del 7 de diciembre de 2020 al 1 de enero de 2021, y se dividió a los participantes en dos grupos:

Grupo A (control): este grupo utilizó el sistema de recomendaciones existente.

Grupo B: este grupo utilizó el nuevo sistema de recomendaciones.

La prueba se realizó en la región de la UE y se incluyó al 15% de los nuevos usuarios.

### Instrucciones para completar la tarea

- Describe los objetivos del estudio.
- Explora los datos:
    - ¿Es necesario convertir los tipos?
    - ¿Hay valores ausentes o duplicados? Si es así, ¿cómo los caracterizarías?
- Lleva a cabo el análisis exploratorio de datos:
    - Estudia la conversión en las diferentes etapas del embudo.
    - ¿El número de eventos por usuario está distribuido equitativamente entre las muestras?
    - ¿Hay usuarios que están presentes en ambas muestras?
    - ¿Cómo se distribuye el número de eventos entre los días?
    - ¿Hay alguna peculiaridad en los datos que hay que tener en cuenta antes de iniciar la prueba A/B?
- Evaluar los resultados de la prueba A/B:
    - ¿Qué puedes decir sobre los resultados de la prueba A/B?
    - Utiliza una prueba z para comprobar la diferencia estadística entre las proporciones
- Describe tus conclusiones con respecto a la etapa EDA y los resultados de la prueba A/B

## Objetivos

Realizar una prueba A/B para verificar si aumenta al menos un 10% en cada etapa del embudo de conversión __product_page → product_card → purchase__ dentro de los siguientes 14 días posteriores a la inscripción.


## Explorar Datos

In [1]:
# Importación de librerías

import pandas as pd
import scipy.stats as stats
import datetime as dt
import numpy as np
import matplotlib.pyplot as plt

## Análisis Exploratorio de Datos

In [2]:
# Carga de datos
project_mkt = pd.read_csv('final_ab_project_marketing_events_us.csv', sep=',')
new_users = pd.read_csv('final_ab_new_users_upd_us.csv', sep=',')
events = pd.read_csv('final_ab_events_upd_us.csv', sep=',')
participants = pd.read_csv('final_ab_participants_upd_us.csv', sep=',')

## Descripción de datasets
- `project_mkt`: el calendario de eventos de marketing para 2020
- `new_users`: todos los usuarios que se registraron en la tienda en línea desde el 7 hasta el 21 de diciembre de 2020
- `events`: todos los eventos de los nuevos usuarios en el período comprendido entre el 7 de diciembre de 2020 y el 1 de enero de 2021
- `participants`: tabla con los datos de los participantes de la prueba


Se muestra la información preliminar de cada dataset

### project_mkt
Estructura `project_mkt`

- `name`: el nombre del evento de marketing
- `regions`: regiones donde se llevará a cabo la campaña publicitaria
- `start_dt`: fecha de inicio de la campaña
- `finish_dt`: fecha de finalización de la campaña

In [10]:
# mostrar infromación preliminar de los datos
project_mkt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14 entries, 0 to 13
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   name       14 non-null     object
 1   regions    14 non-null     object
 2   start_dt   14 non-null     object
 3   finish_dt  14 non-null     object
dtypes: object(4)
memory usage: 580.0+ bytes


El data set no tiene datos ausentes, los nombres de columna están en snake_case y se debe convertir el tipo de datos alusivo a fechas. Verificar adicionalmente los duplicados

In [16]:
project_mkt['start_dt'] = pd.to_datetime(project_mkt['start_dt']) # conversión a datetime
project_mkt['finish_dt'] = pd.to_datetime(project_mkt['finish_dt']) # conversión a datetime

# Obtener días transcurridos de la campaña
project_mkt['duration'] = (project_mkt['finish_dt'] - project_mkt['start_dt']).dt.days
project_mkt.head() # mostrar el nuevo dataframe (5 primeros registros)

,name,regions,start_dt,finish_dt,duration
0,Christmas&New Year Promo,"EU, N.America",2020-12-25,2021-01-03,9
1,St. Valentine's Day Giveaway,"EU, CIS, APAC, N.America",2020-02-14,2020-02-16,2
2,St. Patric's Day Promo,"EU, N.America",2020-03-17,2020-03-19,2
3,Easter Promo,"EU, CIS, APAC, N.America",2020-04-12,2020-04-19,7
4,4th of July Promo,N.America,2020-07-04,2020-07-11,7


### new_users

In [11]:
new_users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58703 entries, 0 to 58702
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   user_id     58703 non-null  object
 1   first_date  58703 non-null  object
 2   region      58703 non-null  object
 3   device      58703 non-null  object
dtypes: object(4)
memory usage: 1.8+ MB


In [12]:
events.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 423761 entries, 0 to 423760
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   user_id     423761 non-null  object 
 1   event_dt    423761 non-null  object 
 2   event_name  423761 non-null  object 
 3   details     60314 non-null   float64
dtypes: float64(1), object(3)
memory usage: 12.9+ MB


In [9]:
participants.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14525 entries, 0 to 14524
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   user_id  14525 non-null  object
 1   group    14525 non-null  object
 2   ab_test  14525 non-null  object
dtypes: object(3)
memory usage: 340.6+ KB
None 



## Test A/B

## Conclusiones